 ### Simulation of Impulse Train Sampling
$x_p(t) = x(t)p(t)$

 p(t) : sampling function <br>
 T: sampling period <br> 
 $\omega _s = 2 \pi / T $ : sampling frequency = fundamental frequency of p(t)

In [8]:
import math
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from commpy.filters import rcosfilter
from scipy import signal
from scipy.fft import fft, fftfreq, fftshift

from ipywidgets import interact
import ipywidgets as widgets

In [16]:
# With impulse train

f = 5  #Frequency of the input signal
n = 10000
t = np.linspace(0,1,n)
x = 5*np.sin(2*np.pi*f*t)   # input signal
x_f = fft(x)                # frequency domain of input signal
x_f = fftshift(x_f)
f = fftfreq(n, 1/n)
f = fftshift(f)
fs = 20 #[48, 100, 200, 500]    sampling frequency 
A = np.arange(0,n,n//fs) # having fs samples from [0 to n]
p = signal.unit_impulse(n, A) # Impulse train of frequency fs (sampling rate of fs)
s = x*p  # sampled signal

p_f = fft(p)
s_f = fft(s)
s_f = fftshift(s_f)
p_f = fftshift(p_f)

fig = make_subplots(rows = 3, cols = 2, subplot_titles=("original", "original", "impulse train", "impulse train", "sampled signal", "sampled signal"))
#fig = go.Figure()
fig.update_layout(title_text="sampling")


# time domain
fig.add_scatter(x = t[:n//2], y = x, row = 1, col = 1)
fig.add_scatter(x = t[:n//2], y = p, mode = "lines", row = 2, col = 1)
fig.add_scatter(x = t[:n//2], y = s, row = 3, col = 1)
fig.update_xaxes(title_text="time", row=3, col=1)

# #frequency domain
fig.add_scatter(x = f, y = abs(x_f), row = 1, col = 2)
fig.add_scatter(x = f, y = abs(p_f), mode = "lines", row = 2, col = 2)
fig.add_scatter(x = f, y = abs(s_f), row = 3, col = 2)
fig.update_xaxes(title_text="freq", row=3, col=2)


fig.update_xaxes(range=[-100,100], row =1, col = 2)
fig.update_xaxes(range=[-100,100], row =2, col = 2)
fig.update_xaxes(range=[-100,100], row =3, col = 2)

fig.layout.update(showlegend=False)

fig

In [5]:
# With square function and duty cycle

f = 50  # Frequency of the original signal
n = 1000*10**2   #number of points for the original signal
t = np.linspace(0,1,n)  #time range
x = 5*np.sin(2*np.pi*f*t)  #original signal
x_f = fft(x)    # original signal in frequency domain 
f = fftfreq(n, 1/n)  #frequency axis


fig = make_subplots(rows = 3, cols = 2, subplot_titles=("original", "original", "impulse train", "impulse train", "sampled signal", "sampled signal"))

fig.update_layout(title_text="sampling")

# time domain
fig.add_scatter(x = t[:n//2], y = x, row = 1, col = 1)
fig.add_scatter(x = t[:n//2], mode = "lines", row = 2, col = 1)
fig.add_scatter(x = t[:n//2], row = 3, col = 1)
fig.update_xaxes(title_text="time", row=3, col=1)

# #frequency domain
fig.add_scatter(x = f, y = abs(x_f), row = 1, col = 2)
fig.add_scatter(x = f, mode = "lines", row = 2, col = 2)
fig.add_scatter(x = f, row = 3, col = 2)
fig.update_xaxes(title_text="freq", row=3, col=2)

fig.layout.update(showlegend=False)
figw = go.FigureWidget(fig)

@interact (fs = [48, 100, 200,500, 1000], duty = [0.001, 0.1, 0.3, 0.5, 0.7])

def update(fs = 200, duty = 0.5):
    p = 0.5*signal.square( t*2*np.pi*fs, duty) + 0.5
    s = x*p
    with figw.batch_update():
        figw.data[1].y = p
        figw.data[2].y = s
        figw.data[4].y = abs(fft(p))/p.size
        figw.data[5].y = abs(fft(s))
figw


interactive(children=(Dropdown(description='fs', index=2, options=(48, 100, 200, 500, 1000), value=200), Dropd…

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '291e6024-0313-4826-b5c4-de6eea0f6591',
 …


#### Background
To convert analog signal (continuous) to digital signal (discrete) we go through several process. 

First we pass the signal through a filter (usually LP) to remove any frequency region of the signal that might be out of our interest. Or in other words, we make the signal to a finite bandwidth. <br>
Than Sampling is done to convert continuous time to discrete time. <br>
Afterwords, we Quantize the signal to convert continuous amplitude to discrete amplitude values. <br>

The general process is <br>
Continuous signal input $\rightarrow$ LP filter $\rightarrow$ Sampling $\rightarrow$ Quantization


When sampling a signal, there is a condition we have to keep so that the signal can capture all the information in the continuous time. <br>
This is called the **Sampling Theorem** and it specifies the minimum *sampling rate* at which a continuous time signal need to be uniformely sampled so that the original signal can be completely reconstructed by the samples alone.

This theorem states that, <br>

If a function have no frequency higher than B, the sampling rate should be higher 2B (nyquist rate)


Sampling is done by multiplying a signal in time domain with an impulse train function. <br>
Impulse train function in time domain gives an impulse train in frequency domain too. <br>
So the output signal frequency response creates a shifted version of the input signal's frequency response, shifted uniformly every fs[Hz] when fs is the impulse train sampling frequency. <br>
When we assume if the input signal as real, the signal would be symmetric in frequency domain. Therefore, if the input signal had a maximum frequency of B, the sampling frequency should be twice the maximum frequency (2B) to avoide alising.